In [2]:
from langchain_anthropic import ChatAnthropic
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import HumanMessage
import os


In [3]:
from bsm_utils import BSMUtils
from langchain.tools import Tool
import json
from dotenv import load_dotenv

In [4]:
import bsm_utils
print(dir(bsm_utils))

['BSMUtils', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'norm', 'np', 'pd', 'tool']


In [5]:
load_dotenv()

True

In [ ]:
bsm_tool = Tool(
    name="BSM_Calculator",
    func=BSMUtils.black_scholes,
    description="Calculates Black-Scholes-Merton option price. Inputs: option_type, S, K, T, r, sigma."
)

csv_reader_tool = Tool(
    name="CSV_Reader",
    func=BSMUtils.read_csv_tool,
    description="Reads a CSV file and returns the first few rows. Inputs: filepath (str), nrows (int, optional)."
)
###Should we convert csv into json?###

llm = ChatAnthropic(
    api_key=os.getenv("ANTHROPIC_API_KEY"),
    model="claude-3-5-sonnet-20240620", 
)

# Create the agent
memory = MemorySaver()
tools = [bsm_tool, csv_reader_tool]
agent_executor = create_react_agent(llm, tools, checkpointer=memory)

inputs = {"messages": [HumanMessage(content="Calculates Black-Scholes-Merton option price. Inputs: Read the first 5 rows from [dummy_options.csv] using the CSV Reader tool.")]}
config = {"configurable": {"thread_id": "thread_1"}}
# Run the agent with a prompt and thread_id
response = agent_executor.invoke(inputs, config=config)

print(response)

BadRequestError: Error code: 400 - {'type': 'error', 'error': {'type': 'invalid_request_error', 'message': "tools.0.custom.name: String should match pattern '^[a-zA-Z0-9_-]{1,128}$'"}, 'request_id': 'req_011CTdqmG8GTSs8v5DFss9WY'}